In [2]:
import pandas as pd
import numpy as np
#import warnings
#warnings.filterwarnings('ignore')
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression

In [3]:
#carregar dataframes
df_test = pd.read_csv('C:/Users/bodel/Documents/coding/codenation_data_science/test.csv')
df_train = pd.read_csv('C:/Users/bodel/Documents/coding/codenation_data_science/train.csv')
#remover coluna com indices ~estranhos
df_train.drop(df_train.columns[0], axis=1, inplace=True)

In [4]:
#remover linhas de candidatos que não compareceram a/foram eliminados de alguma das provas
#DATASET TREINO
df_train.drop(df_train[df_train.TP_PRESENCA_CN!=1].index, inplace=True)
df_train.drop(df_train[df_train.TP_PRESENCA_CH!=1].index, inplace=True)
df_train.drop(df_train[df_train.TP_PRESENCA_LC!=1].index, inplace=True)
df_train.drop(df_train[df_train.TP_PRESENCA_MT!=1].index, inplace=True)
#remover linhas de pessoas com redações problemáticas
df_train.drop(df_train[df_train.TP_STATUS_REDACAO!=1].index, inplace=True)

In [5]:
#inserir coluna a ser predita no dataset de teste pra não apagar os dados na de treino 
df_test.insert(2,"NU_NOTA_MT", 0)
train_idx = df_train.columns
test_idx = df_test.columns
#apagar as colunas que estão no dataset de treino e não no de teste 
df_train.drop(train_idx.difference(test_idx,sort=False), axis=1,inplace=True)
#ajustar os índices
df_train.reset_index(drop=True, inplace=True)

In [6]:
#remover algumas colunas não úteis:
#SG_UF_RESIDENCIA é redundante com CO_UF_RESIDENCIA
#TP_PRESENCA(..) foi usado para filtrar linhas por presença/eliminação, não será mais útil
df_train.drop(['SG_UF_RESIDENCIA','TP_PRESENCA_CN','TP_PRESENCA_CH','TP_PRESENCA_LC', 'TP_STATUS_REDACAO'], axis=1,inplace=True)
df_test.drop(['SG_UF_RESIDENCIA','TP_PRESENCA_CN','TP_PRESENCA_CH','TP_PRESENCA_LC','TP_STATUS_REDACAO'], axis=1,inplace=True)
#remover colunas referentes a cor da prova/língua estrangeira - zero relevância já que não tem sequencia do gabarito no dataset de teste
df_train.drop(['CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC', 'CO_PROVA_MT', 'TP_LINGUA'], axis=1,inplace=True)
df_test.drop(['CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC', 'CO_PROVA_MT', 'TP_LINGUA'], axis=1,inplace=True)
#remover colunas de atendimento especializado por motivo não cognitivo
df_train.drop(['IN_IDOSO', 'IN_GESTANTE', 'IN_SABATISTA'], axis=1,inplace=True)
df_test.drop(['IN_IDOSO', 'IN_GESTANTE', 'IN_SABATISTA'], axis=1,inplace=True)

In [7]:
df_train.describe()

,CO_UF_RESIDENCIA,NU_IDADE,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,TP_DEPENDENCIA_ADM_ESC,...,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO
count,9860.000000,9860.000000,9860.000000,9860.000000,9860.000000,9860.00000,9860.000000,3551.000000,9860.000000,3551.000000,...,9860.000000,9860.000000,9860.000000,9860.000000,9860.000000,9860.000000,9860.000000,9860.000000,9860.000000,9860.000000
mean,30.560953,20.525862,2.168966,1.045233,1.803753,1.73428,1.415213,1.129823,0.147870,2.294847,...,474.894310,531.788732,518.008611,483.634503,122.761866,114.690061,109.475659,115.931542,79.319980,542.179108
std,9.809404,6.279768,1.005135,0.255545,0.826902,2.88892,0.594129,0.486190,0.354989,0.739679,...,71.006222,72.953387,67.931015,99.783008,23.532664,29.198828,30.381720,28.770965,41.842657,131.856441
min,11.000000,13.000000,0.000000,0.000000,1.000000,0.00000,1.000000,1.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,40.000000,0.000000,0.000000,0.000000,40.000000
25%,23.000000,17.000000,1.000000,1.000000,1.000000,0.00000,1.000000,1.000000,0.000000,2.000000,...,420.900000,482.700000,470.975000,409.600000,120.000000,100.000000,80.000000,100.000000,40.000000,460.000000
50%,31.000000,18.000000,3.000000,1.000000,2.000000,0.00000,1.000000,1.000000,0.000000,2.000000,...,461.300000,533.700000,522.400000,462.100000,120.000000,120.000000,120.000000,120.000000,80.000000,540.000000
75%,35.000000,21.000000,3.000000,1.000000,2.000000,2.00000,2.000000,1.000000,0.000000,2.000000,...,516.400000,583.200000,565.800000,539.125000,140.000000,120.000000,120.000000,120.000000,120.000000,600.000000
max,53.000000,66.000000,5.000000,4.000000,4.000000,10.00000,4.000000,3.000000,1.000000,4.000000,...,806.400000,807.000000,763.600000,952.000000,200.000000,200.000000,200.000000,200.000000,200.000000,1000.000000


In [8]:
#remover colunas com _muitos_ dados faltantes
df_train.drop(['TP_ENSINO', 'TP_DEPENDENCIA_ADM_ESC'], axis=1,inplace=True)
df_test.drop(['TP_ENSINO', 'TP_DEPENDENCIA_ADM_ESC'], axis=1,inplace=True)

In [9]:
#transformar dados não numéricos em numéricos:
df_train.TP_SEXO.replace(['F', 'M'], [1, 0], inplace=True)
df_test.TP_SEXO.replace(['F', 'M'], [1, 0], inplace=True)
df_train.Q001.replace(['A','B','C','D','E','F','G','H'], [0,1,2,3,4,5,6,7], inplace=True)
df_test.Q001.replace(['A','B','C','D','E','F','G','H'], [0,1,2,3,4,5,6,7], inplace=True)
df_train.Q002.replace(['A','B','C','D','E','F','G','H'], [0,1,2,3,4,5,6,7], inplace=True)
df_test.Q002.replace(['A','B','C','D','E','F','G','H'], [0,1,2,3,4,5,6,7], inplace=True)
df_train.Q006.replace(['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q'], [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16], inplace=True)
df_test.Q006.replace(['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q'], [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16], inplace=True)
df_test.Q024.replace(['A','B','C','D','E'], [0,1,2,3,4], inplace=True)
df_train.Q024.replace(['A','B','C','D','E'], [0,1,2,3,4], inplace=True)
df_test.Q025.replace(['A','B'], [0,1], inplace=True)
df_train.Q025.replace(['A','B'], [0,1], inplace=True)
df_test.Q026.replace(['A','B','C'], [0,1,2], inplace=True)
df_train.Q026.replace(['A','B','C'], [0,1,2], inplace=True)
#preencher coluna Q027
df_train.Q027.fillna("N",inplace=True)
df_test.Q027.fillna("N",inplace=True)
#continuando conversão
df_train.Q027.replace(['A','B','C','D','E','F','G','H','I','J','K','L','M','N'], [0,1,2,3,4,5,6,7,8,9,10,11,12,13], inplace=True)
df_test.Q027.replace(['A','B','C','D','E','F','G','H','I','J','K','L','M','N'], [0,1,2,3,4,5,6,7,8,9,10,11,12,13], inplace=True)
df_test.Q047.replace(['A','B','C','D','E'], [0,1,2,3,4], inplace=True)
df_train.Q047.replace(['A','B','C','D','E'], [0,1,2,3,4], inplace=True)

In [10]:
#juntar atendimento especializado por deficiência e excluir colunas
df_train['DESABILIDADE'] = (
df_train['IN_BAIXA_VISAO'])+(
2*df_train['IN_CEGUEIRA'])+(
3*df_train['IN_SURDEZ'])+(
4*df_train['IN_DISLEXIA'])+(
5*df_train['IN_DISCALCULIA'])

df_test['DESABILIDADE'] = (
df_test['IN_BAIXA_VISAO'])+(
2*df_test['IN_CEGUEIRA'])+(
3*df_test['IN_SURDEZ'])+(
4*df_test['IN_DISLEXIA'])+(
5*df_test['IN_DISCALCULIA'])

df_train.drop(['IN_BAIXA_VISAO', 'IN_CEGUEIRA',  'IN_SURDEZ',  'IN_DISLEXIA',  'IN_DISCALCULIA'], axis=1,inplace=True)
df_test.drop(['IN_BAIXA_VISAO', 'IN_CEGUEIRA',  'IN_SURDEZ',  'IN_DISLEXIA',  'IN_DISCALCULIA'], axis=1,inplace=True)

In [11]:
df_train.describe()

,CO_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,IN_TREINEIRO,NU_NOTA_CN,...,NU_NOTA_REDACAO,Q001,Q002,Q006,Q024,Q025,Q026,Q027,Q047,DESABILIDADE
count,9860.000000,9860.000000,9860.000000,9860.000000,9860.000000,9860.000000,9860.00000,9860.000000,9860.000000,9860.000000,...,9860.000000,9860.000000,9860.000000,9860.000000,9860.000000,9860.000000,9860.000000,9860.000000,9860.000000,9860.000000
mean,30.560953,20.525862,0.620385,2.168966,1.045233,1.803753,1.73428,1.415213,0.147870,474.894310,...,542.179108,3.077181,3.216126,3.136207,0.708824,0.585903,0.594625,9.371704,0.470284,0.002738
std,9.809404,6.279768,0.485316,1.005135,0.255545,0.826902,2.88892,0.594129,0.354989,71.006222,...,131.856441,1.991061,1.744580,2.927083,0.748737,0.492590,0.787538,4.754173,1.106844,0.088332
min,11.000000,13.000000,0.000000,0.000000,0.000000,1.000000,0.00000,1.000000,0.000000,0.000000,...,40.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,23.000000,17.000000,0.000000,1.000000,1.000000,1.000000,0.00000,1.000000,0.000000,420.900000,...,460.000000,1.000000,2.000000,1.000000,0.000000,0.000000,0.000000,5.000000,0.000000,0.000000
50%,31.000000,18.000000,1.000000,3.000000,1.000000,2.000000,0.00000,1.000000,0.000000,461.300000,...,540.000000,3.000000,3.000000,2.000000,1.000000,1.000000,0.000000,13.000000,0.000000,0.000000
75%,35.000000,21.000000,1.000000,3.000000,1.000000,2.000000,2.00000,2.000000,0.000000,516.400000,...,600.000000,4.000000,4.000000,4.000000,1.000000,1.000000,1.000000,13.000000,0.000000,0.000000
max,53.000000,66.000000,1.000000,5.000000,4.000000,4.000000,10.00000,4.000000,1.000000,806.400000,...,1000.000000,7.000000,7.000000,16.000000,4.000000,1.000000,2.000000,13.000000,4.000000,5.000000


In [12]:
#correlação das colunas que restaram com as notas de matemática
notas = df_train.NU_NOTA_MT
df_train.corrwith(notas, axis=0)


CO_UF_RESIDENCIA    0.108458
NU_IDADE           -0.058702
TP_SEXO            -0.187182
TP_COR_RACA        -0.159643
TP_NACIONALIDADE   -0.036261
TP_ST_CONCLUSAO    -0.084608
TP_ANO_CONCLUIU     0.028189
TP_ESCOLA           0.070282
IN_TREINEIRO       -0.023908
NU_NOTA_CN          0.584474
NU_NOTA_CH          0.528859
NU_NOTA_LC          0.490529
NU_NOTA_MT          1.000000
NU_NOTA_COMP1       0.336469
NU_NOTA_COMP2       0.356981
NU_NOTA_COMP3       0.369217
NU_NOTA_COMP4       0.367265
NU_NOTA_COMP5       0.343410
NU_NOTA_REDACAO     0.413288
Q001                0.168297
Q002                0.211814
Q006                0.411127
Q024                0.324579
Q025                0.248326
Q026               -0.014523
Q027               -0.007210
Q047                0.352508
DESABILIDADE       -0.001240
dtype: float64

In [13]:
#selecionar colunas com maior correlação com notas de matemática
X = df_train[['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_REDACAO', 'Q006', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4']]
Y = df_train.NU_NOTA_MT
regressor = LinearRegression()
scores = []
#validação cruzada para ter noção do r^2, cabe tratamento de outlier para melhorar regressão linear
kfold = KFold(n_splits=3, shuffle=True, random_state=42)
for i, (train, test) in enumerate(kfold.split(X, Y)):
 regressor.fit(X,Y)
 score = regressor.score(X, Y)
 scores.append(score)
print(scores)

[0.4230068591153992, 0.4230068591153992, 0.4230068591153992]


In [14]:
#regressão linear com todos os dados do dataframe de treino, printando coeficientes da reta predita
regressor.fit(X, Y)
['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_REDACAO', 'Q006', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4']
print(regressor.coef_, regressor.intercept_)

[ 0.48139974  0.2128507   0.17897479  0.02916423  5.50303614 -0.06252532
  0.05146869] 16.925681454255653


In [20]:
#predição das notas de matemática com os dados do dataset de treino e coeficientes calculados anteriormente 
df_test['NU_NOTA_MT'] = (
    16.925681) + (
    0.48139974*df_test['NU_NOTA_CN']) + (
    0.2128507*df_test['NU_NOTA_CH']) + (
    0.17897479*df_test['NU_NOTA_LC'])+ (
    0.02916423*df_test['NU_NOTA_REDACAO'])+ (
    5.50303614*df_test['Q006'])+ (
    -0.06252532*df_test['NU_NOTA_COMP3']) + (
    0.05146869*df_test['NU_NOTA_COMP4'])

In [21]:
#criar novo dataframe com as colunas de numero de inscrição e nota predita
new_df = pd.DataFrame(data=df_test, columns=['NU_INSCRICAO','NU_NOTA_MT'])
#arredondar para uma casa decimal
new_df.round(1)

,NU_INSCRICAO,NU_NOTA_MT
0,73ff9fcc02f0a99919906c942c2e1a1042cdcf98,435.8
1,71a95f9f1b91a82c65ad94abbdf9f54e6066f968,434.4
2,b38a03232f43b11c9d0788abaf060f7366053b6d,573.6
3,70b682d9a3636be23f6120fa9d6b164eb3c6002d,NaN
4,715494628a50142ce8cb17191cfe6d0f3cae0934,557.7
...,...,...
4571,dac0f22429c7f8e3931d0abaf5dfc8e5c772a48b,447.8
4572,a75fa8770257e7c9368d059fe53d9ef431f4bdef,456.6
4573,655fa6306720ff16e825903b5422a46608a77545,604.8
4574,1f4bc3e3d56212d500625bfe8ac78ccff4362293,447.2


In [22]:
#exportar csv do novo data frame
new_df.to_csv('C:/Users/bodel/Documents/coding/codenation_data_science/answer.csv', index=False)